## <font color='#eb3483'>Evaluating Regression models</font>

 Evaluating models is an essential step in doing Machine Learning for two reasons:

- So we can evaluate and find the model that performs best.
- So we can calculate the expected margin of error in our predictions.

We are going to look at various error metrics that judge how well regression models make predictions. 

Before we begin, let's load the necessary libraries for data manipulation.

In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(6,6)}) 

import warnings
warnings.simplefilter("ignore")

We're still using the boston housing dataset from the `sklearn` library.

In [5]:
from sklearn import datasets

In [6]:
boston = datasets.load_boston()
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

We split data into training and testing sets, fit the model on the training set and made predictions on the test set. 

P.S to learn more about the `random_state` parameter, see [this Stack Overflow answer](https://stackoverflow.com/a/28069274).

In [7]:
from sklearn.model_selection import train_test_split

In [11]:
independent_variables = boston["feature_names"] # X variable names
target = "target" # y variable name

In [9]:
df = pd.DataFrame(boston["data"], columns=boston["feature_names"])
df["target"] = boston.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [10]:
# How many data points and how many features ?
df.shape

(506, 14)

In [15]:
X = df[independent_variables]    # numpy array
X = X.drop('B', axis=1)          # dropping this column - hells no!
y = df[target]                   # numpy array
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state= 13)

In [16]:
# Stratied split

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test',  X_test.shape)
print('y_test',  y_test.shape)

X_train (339, 12)
y_train (339,)
X_test (167, 12)
y_test (167,)


We instantiate the model using the `LinearRegression` class.

In [17]:
from sklearn.linear_model import LinearRegression 

In [18]:
# this is an object called model with the the class of LinearRegression
model = LinearRegression() # instantiate the model

In [19]:
model.fit(X=X_train, y=y_train) # fit the model
predictions = model.predict(X_test) # make predictions

In [20]:
# see predictions
for y, y_pred in list(zip(y_test, predictions))[:5]:
    print("Real value: {:.3f} Estimated value: {:.5f}".format(y, y_pred))

Real value: 12.000 Estimated value: 10.61002
Real value: 15.200 Estimated value: 19.54526
Real value: 21.000 Estimated value: 20.59385
Real value: 24.000 Estimated value: 30.40627
Real value: 19.400 Estimated value: 23.26189


![title](media/train_test_split.png)
![title](media/train_test.png)

## We now look at metrics to evaluate our predictions.

This allows us to get an indication of how good or how poor our predictions actually were - without just eyeballing some visualisations :)

Model evaluation metrics in `scikit-learn` are available under the module `metrics`

In [21]:
# import metrics from sklearn
from sklearn import metrics

### <font color='#eb3483'>Mean Absolute Error (MAE)</font>

The Mean Absolute Error (MAE) is defined as:

$$\frac{1}{n}\sum_{i=1}^{n}|y_i -\hat{y}_i|$$

Basically the differences between the real values of the target variable and the predictions in absolute value (so turning negative differences into positive ones).

MAE is a robust metric, that means it doesnt change dramatically when there are outliers. The MAE error can be interpreted in the same units of the target variable (so for example, if the target variable is in dollars, the MAE will also be in dollars).

In [22]:
metrics.mean_absolute_error?

In [24]:
mae = metrics.mean_absolute_error(y_test, predictions)
print("The Mean Absolute Error is {:.3f} dollars".format(mae))
#since housing prices are high (couple thousand), the error is relatively small 

The Mean Absolute Error is 3.670 dollars


### <font color='#eb3483'>Mean Squared Error (MSE)</font>

The Mean Squared Error (MSE) is defined as:

$$\frac{1}{n}\sum_{i=1}^{n}(y_i -\hat{y}_i)^2$$

Similar to the MAE, but using the square of the difference between the true target and the prediction. 

MSE gives more weight to larger errors than MAE (is not robust to outliers). For example, let's imagine we are predicting housing prices using the Boston dataset and we have the following observations:

```
observation1: MEDV: 10  MEDV_pred: 15    MSE: (10-15)²=25
observation2: MEDV: 1000 MEDV_pred: 1010 MSE: (1000-1010)²=100 
```

By using MSE we are giving more weight to the error on observation2 than on observation1, even though a 5000`$` error on a 15,000`$` house is a much worse error than a 10,000`$` error on a 1,000,000`$` house.

The MSE is measured in squared units (squared dollars?) which is hard to understand, so there is another metric called Root Mean Squared Error (RMSE) that is just the root of the MSE.

In [19]:
mse = metrics.mean_squared_error(y_test, predictions)
print("The Mean Squared Error is {:.3f} dollars²".format(mse))

The Mean Squared Error is 23.649 dollars²


### <font color='#eb3483'>Root Mean Squared Error (RMSE)</font>

Root Mean Squared Error (RMSE) is just the root of the MSE, and it is measured in the same units as the target variable.

$$\sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i -\hat{y}_i)^2}$$

Similarly to MSE, RMSE is more sensitive than MAE to variations in errors. Here we can see an example of this.

![mae_vs_mse](media/mse_vs_mae.png)

In this example we can see that on the 3 cases MAE remains the same, while RMSE can be equal to MAE or much larger depending on the error distribution.

So in those cases where we care about making big mistakes we can use RMSE. For example, if we are predicting student grades, we might not care that much about individual errors (predicting a 10 when the truth was a 2 is not a big deal), but about the overall performance (and we could use MAE). If we are predicting house values to purchase them, predicting a million dollars when the actual house price is 20,000$ is an error we cant afford!

In [29]:
# sklearn doesnt have rmse, but it's easy to create the metric "manually"
rmse = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print("The Root Mean Squared Error is {:.2f} dollars".format(rmse))

The Root Mean Squared Error is 4.86 dollars


### <font color='#eb3483'>R2 (Coefficient of Determination)</font>

The Coefficient of Determination (R2, pronounced *R-squared*) measures the portion of the variance that can be explained by the model.

R2 ranges from (-1 to 1) (a model explaining all the variance would have an $r^2$ of 1).

[There are many ways to measure $r^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination) , but one of the simplest ones is simply the squared [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)  between the true target and the prediction, squared.

$$r^2=\frac{\sum_{i=1}^n(\hat y_i-\bar y)^2}{\sum_{i=1}^n(y_i-\bar y)^2}.$$

#### <font color='#eb3483'>Adjusted R2</font>

There is an updated version of R2 called **Adjusted R-squared** that takes into consideration model complexity (so it penalizes a complex model versus a simple one).

$$1 - \frac{(1 - R^2)(n-1)}{(n-k-1)}$$

where `n` is the number of observations and `k` is the number of model coefficients.

<!--Contrary to `MAE` and `MSE` the higher the R2 the better.-->

In [27]:
model_r2 = metrics.r2_score(y_test, predictions)
model_r2

0.7035577824232988

In [28]:
pearson_corr = np.corrcoef(y_test, predictions)
model_r2_ = pearson_corr**2
model_r2_

array([[1.        , 0.71381657],
       [0.71381657, 1.        ]])

We can calculate easily the adjusted r² value by hand, to account for model complexity. This value depends on the number of coefficients and imposes a penalty for additional coefficients
<!--(number of coefficients). 
, the regularization lesson goes further into the issue of model complexity.-->

In [23]:
len(model.coef_)

12

In [30]:
n = len(X_test)
k = len(model.coef_)
r2 = model_r2

adjusted_r2_model = 1 - ((1-r2)*(n-1)/(n-k-1))
adjusted_r2_model

0.6804583888458935

We see the adjusted r² of the model is smaller than the original r².

### Model Evaluation

We can now evaluate our models. First we create a dictionary to store the results, and a function to help us evaluate models. 

In [33]:
RESULTS = {}

def evaluate_model(y_true, y_pred):
    """Function to evaluate models, you could add more metrics here!"""
    return {
        "mae": metrics.mean_absolute_error(y_true, y_pred),
    }

In [34]:
# create the model
model_ols = LinearRegression()

In [35]:
# first train and predict without splitting data
model_ols.fit(X=boston['data'], y=boston['target'])
model_ols_preds = model_ols.predict(boston['data'])

# store results
RESULTS["ols"] = evaluate_model(
    boston["target"],
    model_ols_preds,
)

RESULTS

{'ols': {'mae': 3.2708628109003115}}

In [28]:
# train and predict using just training dataset
model_ols.fit(X=X_train, y=y_train)
model_ols_train_preds = model_ols.predict(X_train)

RESULTS["ols_train"] = evaluate_model(
    y_train,
    model_ols_train_preds
)

RESULTS

{'ols': {'mae': 3.2708628109003173}, 'ols_train': {'mae': 3.355531297855312}}

In [29]:
# predict using the test dataset
model_ols_test_preds = model_ols.predict(X_test)
RESULTS["ols_test"] = evaluate_model(
    y_test,
    model_ols_test_preds
)

RESULTS

{'ols': {'mae': 3.2708628109003173},
 'ols_train': {'mae': 3.355531297855312},
 'ols_test': {'mae': 3.6701893264960854}}

In [30]:
pd.DataFrame(RESULTS)

,ols,ols_train,ols_test
mae,3.270863,3.355531,3.670189


We can see that we get we get worse results on the test set than on the training set.

We could just stop here and say *"Our model MAE on the test dataset is 3.67..."*, and we could think everything is ok with this because that is the error on unseen data.

However, this would be a big mistake, why? 

Remember, we have used a specific random state `random_state=13`, what would happen if we use another seed, for example, `random_state=42`?

**Let's run everything again with a diffefent random seed**

In [36]:
RESULTS = {}

X_train, X_test, y_train, y_test = train_test_split(
     boston["data"],   # X
     boston["target"], # y
     test_size=0.33, 
     random_state=42
)

model_ols = LinearRegression()

model_ols.fit(X=X_train, y=y_train)
model_ols_train_preds = model_ols.predict(X_train)
model_ols_test_preds = model_ols.predict(X_test)


RESULTS["ols_train2"] = evaluate_model(
    y_train,
    model_ols_train_preds
)

RESULTS["ols_test2"] = evaluate_model(
    y_test,
    model_ols_test_preds
)

pd.DataFrame(RESULTS)

,ols_train2,ols_test2
mae,3.376419,3.148256


**The test MAE is lower than the train one!!, WHAAAT!!??** How can that even be possible???

Very simple, it just turns out the split generated by using the seed number 42 puts on the test dataset observations that are easier to predict.

## <font color='#eb3483'> Cross-validation </font>

One way to avoid evaluating on one single lucky split is by doing **Cross Validation**. When we do cross validation we simply split the data into **N** partitions, for each partion, we train the model with the remaining N-1 partitions and evaluate on that one. That way we get N evaluation errors trained and evaluated on different data so we don't rely on a single split. We finally make predictions on the test set to get an unbiased view of model performance.

For example, a 5 fold cross validation would look like this:

![title](media/cross_validation.png)

<!--![title](media/grid_search_cross_validation.png)-->

`scikit-learn` has a function `cross_val_score` that evaluates a model doing cross validation.

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
cross_val_score?
#the estimator hyperparameter is the model, X and y are the datasets 

To use `cross_val_score`, we need a model, the independent and dependent variables (X and y). We also have to define an evaluation metric (`scoring` argument) and the number of splits (`cv` argument).

In [39]:
model = LinearRegression()

We can choose any of the scorers defined on `sklearn.metrics.SCORERS` or we can create our own scoring function.

In [40]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

For example, if we want to perform cross validation using the mean absolute error (MAE), we can use the scoring `neg_mean_squared_error`, that is the name of the mean absolute error on the `SCORERS` dictionary above.

**Note**: In scikit-learn, the "bad" scores (i.e. errors) are returned as negative numbers

In [1]:
results_cross_validation = cross_val_score(
    estimator=model, 
    X=boston["data"],
    y=boston["target"],
    scoring="neg_mean_absolute_error", 
    cv=5,
)
#scoring: calculate metrics, chosen from SCORERS list above 

NameError: name 'cross_val_score' is not defined

`cross_val_score` returns the evaluation on the test set for each one of the splits (in this case, 5 splits). Normal number of splits are between 3 and 10

In [45]:
results_cross_validation
#values for each split

array([-2.62190565, -3.90725478, -4.386606  , -5.57073637, -4.76333993])

We can finally calculate the average partition error to get a better estimation of its performance.

In [46]:
# we use abs to get positive MAE values
model_mae = abs(results_cross_validation.mean())
print(f"the average MAE for 5 split cross validation is : {model_mae}")

the average MAE for 5 split cross validation is : 4.249968544192538


We can also define our own cross validation evaluation metric, which is simply a function that expects to receive as arguments the trained estimator, X and y and return the error.

For example, if we want to use rmse as a scoring for cross validation, we can create our own scoring function:

In [47]:
def rmse_cross_val(estimator, X, y):
    y_pred = estimator.predict(X)
    return np.sqrt(metrics.mean_squared_error(y, y_pred))


In [48]:
results_cross_validation_rmse = cross_val_score(
    estimator=model, 
    X=boston["data"],
    y=boston["target"],
    scoring=rmse_cross_val, 
    cv=5, 
)

mean_rmse_cv = results_cross_validation_rmse.mean()
print(f"the average RMSE for 5 split cross validation is : {mean_rmse_cv}")

the average RMSE for 5 split cross validation is : 5.8286589462158345


**A bit on hyperparameters.** The function `cross_val_score` has the argument `cv`. We can adjust its value to improve the model score. 

In [41]:
cross_val_score?

## <font color='#eb3483'>BONUS: The function cross_validate</font>

If we want to get more information about each split, we can use the function `cross_validate` that returns more information besides the test error. It also accepts multiple scoring functions instead of just one. Think of `cross_val_score` as the simplified version of `cross_validate`.

In [51]:
from sklearn.model_selection import cross_validate

scoring_functions = {"mae": "neg_mean_absolute_error", "rmse": rmse_cross_val, "r2": "r2"}

scores = cross_validate(
    model,                # estimator
    boston["data"],            # X
    boston["target"],          # y
    scoring=scoring_functions, # a single scorer or a dict with multiple scoring functions
    cv=10,                      # number of partitions
    return_train_score=True    # return the training error, not only the test error
)

In [52]:
results_df = pd.DataFrame(scores)

In [53]:
results_df

,fit_time,score_time,test_mae,train_mae,test_rmse,train_rmse,test_r2,train_r2
0,0.015994,0.002481,-2.206868,-3.413087,3.047449,4.833552,0.733761,0.738813
1,0.003992,0.002990,-2.896809,-3.385031,3.761819,4.783517,0.473073,0.747091
2,0.013702,0.001995,-2.786730,-3.429977,3.751481,4.818306,-1.006315,0.743834
3,0.014513,0.008929,-4.598478,-3.105423,5.933542,4.557598,0.641140,0.717213
4,0.014771,0.018592,-4.109865,-3.272898,5.646691,4.619029,0.547660,0.741724
5,0.015638,0.001885,-3.564692,-3.276768,4.453749,4.729027,0.736403,0.704314
6,0.022238,0.003099,-2.669667,-3.404997,3.153929,4.829826,0.378284,0.746070
7,0.011809,0.003603,-9.656378,-2.612918,12.975954,3.458207,-0.129227,0.838756
8,0.004756,0.002000,-5.022725,-3.191380,5.773192,4.646105,-0.768432,0.736517
9,0.002455,0.003948,-2.537253,-3.383958,3.310651,4.815500,0.418943,0.742119


We get results for each one of the partitions:
- fit time, how long it takes to train the model
- score time, how long it takes to predict
- test and train scores for each one of the scoring functions

Now we can calculate the averages for all the partitions:

In [54]:
pd.DataFrame(scores).mean()

fit_time      0.011987
score_time    0.004952
test_mae     -4.004947
train_mae    -3.247644
test_rmse     5.180846
train_rmse    4.609067
test_r2       0.202529
train_r2      0.745645
dtype: float64